<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/345_WDO_GapDetection_Utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a **very strong and thoughtful skill gap layer**. You’ve managed to do something that many workforce systems fail at:
you connect **skills, roles, and automation risk** into a single, coherent decision surface — without hiding logic in a model.


---

# Workforce Development Orchestrator — Skill Gap Detection Utilities

This module is responsible for identifying **where workforce capability gaps exist** and determining **which gaps matter most right now**. It is the bridge between automation risk and actionable workforce investment.

Importantly, skill gaps here are **not treated as abstract deficiencies**. They are contextualized by:

* Role requirements
* Future role evolution
* Automation risk exposure
* Organizational urgency

This is what allows the orchestrator to move from *analysis* to *strategy*.

---

## 1. Skill Gaps Are Detected Relative to Roles, Not in Isolation

The `detect_skill_gaps_for_employee` function evaluates an employee’s skills **in the context of their role**, not against a generic skills checklist.

It explicitly compares:

* Current skills
* Required role skills
* Future role skills

This reflects how real organizations think:

> Skills only matter relative to the work someone is expected to do — now and next.

---

## 2. Required vs. Future Skills Are Treated Differently (By Design)

The utility distinguishes between two fundamentally different gap types:

### 🔴 Missing Required Skills

* Classified as high priority by default
* Assigned high urgency scores
* Always treated as role-critical

These represent **immediate execution risk**.

---

### 🟡 Missing Future Skills

* Priority depends on automation exposure
* Urgency increases when roles are at higher automation risk
* Represents **strategic resilience risk**, not failure

This avoids a common pitfall: overreacting to future change while neglecting current needs.

---

## 3. Existing Gap Data Is Respected, Not Overwritten

If a gap already exists in the dataset, it is **merged rather than replaced**.

This allows:

* Human-reviewed gap data to coexist with automated detection
* Incremental enrichment over time
* Gradual transition from manual to automated systems

This is a critical trust-preserving design choice.

---

## 4. Automation Risk Context Is Integrated, Not Bolted On

The `enhance_gap_with_automation_context` function enriches each gap with:

* Role-level automation risk
* Overall role risk score
* Adjusted urgency based on exposure

This creates a **compounding signal**:

> A missing skill in a high-risk role is more urgent than the same gap in a stable role.

Crucially, urgency adjustments are capped — preventing runaway scoring and maintaining calibration.

---

## 5. Organization-Wide Gap Detection Is Deterministic and Scalable

The `detect_all_skill_gaps` function applies the same logic consistently across the entire workforce.

It:

* Uses role lookups instead of inference
* Creates a clean role → risk mapping
* Produces uniform gap artifacts suitable for dashboards and prioritization

This ensures fairness, repeatability, and comparability.

---

## 6. Priority Scoring Is Explicit and Configurable

The `calculate_gap_priority_score` function translates qualitative signals into a **single, auditable score**.

Each component is transparent:

* Automation exposure
* Role criticality
* Future relevance
* Employee urgency proxy

Weights are pulled directly from configuration, meaning:

* Leadership controls prioritization policy
* Sensitivity can be tuned without code changes
* ROI tradeoffs are explicit

This is **policy-driven prioritization**, not model-driven ranking.

---

## Why This Design Works for Leaders

From an executive perspective, this module ensures:

* Skill investments are aligned to real risk
* Urgency is justified, not assumed
* Future planning does not overshadow current execution
* Recommendations can be explained in plain language

From an engineering perspective, it ensures:

* Deterministic behavior
* Easy testing
* Clear extension points
* No hidden coupling between risk and learning logic

---

## Architectural Takeaway

This module demonstrates a critical principle of responsible workforce AI:

> **Not all skill gaps are equal — and urgency must be earned, not assumed.**

By grounding skill gap detection in role context, automation risk, and configurable policy weights, the orchestrator produces recommendations that leaders can confidently act on.




In [ ]:
"""Skill gap detection utilities for Workforce Development Orchestrator

Following the pattern: Utilities implement, nodes orchestrate.
These utilities detect and analyze skill gaps for employees.
"""

from typing import Dict, List, Any, Optional
from config import WorkforceDevelopmentOrchestratorConfig


def detect_skill_gaps_for_employee(
    employee: Dict[str, Any],
    role: Dict[str, Any],
    skills_lookup: Dict[str, Dict[str, Any]],
    existing_gaps: List[Dict[str, Any]],
    automation_risk_context: Optional[str] = None
) -> List[Dict[str, Any]]:
    """Detect skill gaps for a specific employee"""
    employee_id = employee["employee_id"]
    current_skills = set(employee.get("current_skills", []))
    required_skills = set(role.get("required_skills", []))
    future_skills = set(role.get("future_skills", []))

    gaps = []

    # Check for missing required skills
    missing_required = required_skills - current_skills
    for skill_id in missing_required:
        # Check if gap already exists in data
        existing_gap = next(
            (g for g in existing_gaps if g.get("employee_id") == employee_id and g.get("skill_id") == skill_id),
            None
        )

        gap = {
            "employee_id": employee_id,
            "employee_name": employee.get("name", ""),
            "role_id": role["role_id"],
            "role_name": role.get("role_name", ""),
            "skill_id": skill_id,
            "skill_name": skills_lookup.get(skill_id, {}).get("skill_name", skill_id),
            "gap_type": "missing_required_skill",
            "priority": "high",
            "role_requirement": True,
            "automation_risk_context": automation_risk_context or "unknown",
            "urgency_score": 0.9  # High urgency for required skills
        }

        # Merge with existing gap data if available
        if existing_gap:
            gap.update(existing_gap)

        gaps.append(gap)

    # Check for missing future skills
    missing_future = future_skills - current_skills
    for skill_id in missing_future:
        # Check if gap already exists in data
        existing_gap = next(
            (g for g in existing_gaps if g.get("employee_id") == employee_id and g.get("skill_id") == skill_id),
            None
        )

        gap = {
            "employee_id": employee_id,
            "employee_name": employee.get("name", ""),
            "role_id": role["role_id"],
            "role_name": role.get("role_name", ""),
            "skill_id": skill_id,
            "skill_name": skills_lookup.get(skill_id, {}).get("skill_name", skill_id),
            "gap_type": "missing_future_skill",
            "priority": "medium" if automation_risk_context != "high" else "high",
            "role_requirement": True,
            "automation_risk_context": automation_risk_context or "unknown",
            "urgency_score": 0.7 if automation_risk_context == "high" else 0.5
        }

        # Merge with existing gap data if available
        if existing_gap:
            gap.update(existing_gap)

        gaps.append(gap)

    return gaps


def enhance_gap_with_automation_context(
    gap: Dict[str, Any],
    automation_risk_analysis: List[Dict[str, Any]]
) -> Dict[str, Any]:
    """Enhance gap with automation risk context"""
    role_id = gap.get("role_id")

    # Find automation risk for this role
    role_risk = next(
        (r for r in automation_risk_analysis if r.get("role_id") == role_id),
        None
    )

    if role_risk:
        gap["automation_risk_context"] = role_risk.get("risk_level", "unknown")
        gap["role_risk_score"] = role_risk.get("overall_risk_score", 0.0)

        # Adjust urgency based on automation risk
        if role_risk.get("risk_level") == "high":
            gap["urgency_score"] = min(1.0, gap.get("urgency_score", 0.5) + 0.2)
        elif role_risk.get("risk_level") == "medium":
            gap["urgency_score"] = min(1.0, gap.get("urgency_score", 0.5) + 0.1)

    return gap


def detect_all_skill_gaps(
    employees: List[Dict[str, Any]],
    roles_lookup: Dict[str, Dict[str, Any]],
    skills_lookup: Dict[str, Dict[str, Any]],
    existing_gaps: List[Dict[str, Any]],
    automation_risk_analysis: List[Dict[str, Any]]
) -> List[Dict[str, Any]]:
    """Detect skill gaps for all employees"""
    all_gaps = []

    # Create automation risk lookup by role
    risk_by_role = {
        r.get("role_id"): r.get("risk_level", "unknown")
        for r in automation_risk_analysis
    }

    for employee in employees:
        role_id = employee.get("role_id")
        role = roles_lookup.get(role_id)

        if not role:
            continue

        automation_risk_context = risk_by_role.get(role_id, "unknown")

        gaps = detect_skill_gaps_for_employee(
            employee,
            role,
            skills_lookup,
            existing_gaps,
            automation_risk_context
        )

        # Enhance gaps with automation context
        enhanced_gaps = [
            enhance_gap_with_automation_context(gap, automation_risk_analysis)
            for gap in gaps
        ]

        all_gaps.extend(enhanced_gaps)

    return all_gaps


def calculate_gap_priority_score(
    gap: Dict[str, Any],
    config: WorkforceDevelopmentOrchestratorConfig
) -> float:
    """Calculate priority score for a skill gap"""
    weights = config.gap_priority_weights

    # Automation risk component
    automation_score = 0.0
    if gap.get("automation_risk_context") == "high":
        automation_score = 1.0
    elif gap.get("automation_risk_context") == "medium":
        automation_score = 0.5

    # Role requirement component
    role_requirement_score = 1.0 if gap.get("role_requirement") else 0.0

    # Future skill component
    future_skill_score = 1.0 if gap.get("gap_type") == "missing_future_skill" else 0.5

    # Employee performance component (using urgency score as proxy)
    performance_score = gap.get("urgency_score", 0.5)

    # Weighted sum
    priority_score = (
        automation_score * weights["automation_risk"] +
        role_requirement_score * weights["role_requirement"] +
        future_skill_score * weights["future_skill"] +
        performance_score * weights["employee_performance"]
    )

    return round(priority_score, 2)



# Test skill gap detection utilities

In [ ]:
"""Test skill gap detection utilities

Testing Phase 4: Skill Gap Detection Utilities
Following the pattern: Test utilities before building nodes
"""

from pathlib import Path
from agents.workforce_development_orchestrator.utilities.skill_gap_detection import (
    detect_skill_gaps_for_employee,
    enhance_gap_with_automation_context,
    detect_all_skill_gaps,
    calculate_gap_priority_score
)
from agents.workforce_development_orchestrator.utilities.data_loading import (
    load_employees,
    load_roles,
    load_skills,
    build_roles_lookup,
    build_skills_lookup
)
from agents.workforce_development_orchestrator.utilities.automation_risk import (
    analyze_all_roles_automation_risk,
    build_tasks_by_role,
    build_employees_by_role
)
from agents.workforce_development_orchestrator.utilities.data_loading import (
    load_tasks,
    load_skill_gaps
)
from config import WorkforceDevelopmentOrchestratorConfig


def test_detect_skill_gaps_for_employee():
    """Test detecting skill gaps for a single employee"""
    config = WorkforceDevelopmentOrchestratorConfig()
    data_dir = Path("agents/data")

    # Load data
    employees = load_employees(data_dir)
    roles = load_roles(data_dir)
    skills = load_skills(data_dir)
    roles_lookup = build_roles_lookup(roles)
    skills_lookup = build_skills_lookup(skills)
    existing_gaps = load_skill_gaps(data_dir)

    # Test with E001 (Sarah Chen)
    employee = employees[0]  # E001
    role = roles_lookup[employee["role_id"]]

    gaps = detect_skill_gaps_for_employee(
        employee,
        role,
        skills_lookup,
        existing_gaps,
        "medium"
    )

    assert len(gaps) > 0
    assert all("employee_id" in g for g in gaps)
    assert all("skill_id" in g for g in gaps)
    assert all("gap_type" in g for g in gaps)

    print("✅ test_detect_skill_gaps_for_employee: PASSED")


def test_enhance_gap_with_automation_context():
    """Test enhancing gap with automation risk context"""
    data_dir = Path("agents/data")

    # Load and analyze automation risk
    roles = load_roles(data_dir)
    tasks = load_tasks(data_dir)
    employees = load_employees(data_dir)
    tasks_by_role = build_tasks_by_role(tasks)
    employees_by_role = build_employees_by_role(employees)
    config = WorkforceDevelopmentOrchestratorConfig()

    automation_risk_analysis = analyze_all_roles_automation_risk(
        roles,
        tasks_by_role,
        employees_by_role,
        config
    )

    # Create a test gap
    gap = {
        "employee_id": "E001",
        "role_id": "R001",
        "skill_id": "ai_tools",
        "gap_type": "missing_future_skill",
        "urgency_score": 0.5
    }

    enhanced = enhance_gap_with_automation_context(gap, automation_risk_analysis)

    assert "automation_risk_context" in enhanced
    assert "role_risk_score" in enhanced
    assert enhanced["urgency_score"] >= gap["urgency_score"]

    print("✅ test_enhance_gap_with_automation_context: PASSED")


def test_detect_all_skill_gaps():
    """Test detecting gaps for all employees"""
    config = WorkforceDevelopmentOrchestratorConfig()
    data_dir = Path("agents/data")

    # Load all data
    employees = load_employees(data_dir)
    roles = load_roles(data_dir)
    skills = load_skills(data_dir)
    tasks = load_tasks(data_dir)
    roles_lookup = build_roles_lookup(roles)
    skills_lookup = build_skills_lookup(skills)
    existing_gaps = load_skill_gaps(data_dir)
    tasks_by_role = build_tasks_by_role(tasks)
    employees_by_role = build_employees_by_role(employees)

    # Analyze automation risk
    automation_risk_analysis = analyze_all_roles_automation_risk(
        roles,
        tasks_by_role,
        employees_by_role,
        config
    )

    # Detect all gaps
    all_gaps = detect_all_skill_gaps(
        employees,
        roles_lookup,
        skills_lookup,
        existing_gaps,
        automation_risk_analysis
    )

    assert len(all_gaps) > 0
    assert all("employee_id" in g for g in all_gaps)
    assert all("automation_risk_context" in g for g in all_gaps)

    print("✅ test_detect_all_skill_gaps: PASSED")


def test_calculate_gap_priority_score():
    """Test calculating priority score for gaps"""
    config = WorkforceDevelopmentOrchestratorConfig()

    # High priority gap
    high_gap = {
        "automation_risk_context": "high",
        "role_requirement": True,
        "gap_type": "missing_future_skill",
        "urgency_score": 0.9
    }

    high_score = calculate_gap_priority_score(high_gap, config)
    assert high_score > 0.7

    # Medium priority gap
    medium_gap = {
        "automation_risk_context": "medium",
        "role_requirement": True,
        "gap_type": "missing_required_skill",
        "urgency_score": 0.6
    }

    medium_score = calculate_gap_priority_score(medium_gap, config)
    assert 0.4 <= medium_score <= 0.8

    print("✅ test_calculate_gap_priority_score: PASSED")


if __name__ == "__main__":
    print("=" * 60)
    print("Testing Skill Gap Detection Utilities (Phase 4)")
    print("=" * 60)
    print()

    test_detect_skill_gaps_for_employee()
    test_enhance_gap_with_automation_context()
    test_detect_all_skill_gaps()
    test_calculate_gap_priority_score()

    print()
    print("=" * 60)
    print("✅ All skill gap utility tests passed!")
    print("=" * 60)



In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_008_Workforce_Development_Orchestrator % python3 test_skill_gap_utilities.py
============================================================
Testing Skill Gap Detection Utilities (Phase 4)
============================================================

✅ test_detect_skill_gaps_for_employee: PASSED
✅ test_enhance_gap_with_automation_context: PASSED
✅ test_detect_all_skill_gaps: PASSED
✅ test_calculate_gap_priority_score: PASSED

============================================================
✅ All skill gap utility tests passed!
============================================================
